# Bluetooth Data to MQTT Publisher
This Python script reads data from a Bluetooth module connected to the COM7 port, parses the received data (which includes heart rate, temperature, and stress status), and publishes it to an MQTT broker (broker.emqx.io) with the topic "StressDet092111161173". The parsed data includes heart rate, temperature, and stress status. If the heart rate and temperature readings are valid and the stress status is available, the script publishes this data to the MQTT broker.

In [ ]:
import time
import paho.mqtt.client as mqtt
import serial

incomingPort = 'COM7'  # Bluetooth Module COM port 
baudRate = 9600
incomingSer = serial.Serial(incomingPort, baudRate) #Port for receiving Data

# MQTT Broker details ------------------------>
brokerAddress = "broker.emqx.io"
mqttPort = 1883
mqttTopic = "StressDet092111161173"  # Topic to publish the data

# MQTT Client setup --------------------------->
mqttc = mqtt.Client(client_id="mqttx_241fee99", clean_session=True, callback_api_version=mqtt.CallbackAPIVersion.VERSION2)
mqttc.connect(brokerAddress, mqttPort)
mqttc.loop_start()

def parseAndpublish(data):
    # Parse the incoming data
    data_parts = data.split(", ")
    
    # Initialize default values
    heart_rate = "0"
    temperature = "0"
    stress_status = "NO DATA"
    
    # Extract values if available
    for part in data_parts:
        if "BPM" in part:
            bpm_split = part.split("BPM=")
            if len(bpm_split) > 1:
                if ('%' not in bpm_split[1]):
                    heart_rate = bpm_split[1]
        if "Temperature" in part:
            temp_split = part.split("Temperature: ")
            if len(temp_split) > 1:
                temperature = temp_split[1]
        if "STRESS" in part:
            stress_status = "STRESS"
        if "NO STRESS" in part:
            stress_status = "NO STRESS"
            
    heart_rate=float(heart_rate)
    temperature=float(temperature.replace("°C", "") if temperature.replace("°C", "") else 0)
    print("\n Sensor Message: ",data)
    if(heart_rate >40 and temperature > 0 and stress_status != 'NO DATA'): #So that incorrect readings are not published
        message = f"Heart Rate: {heart_rate}, Temperature: {temperature}, Stress: {stress_status}"
        print("\n Published Message: ",message)
        # Publish the message to MQTT Broker
        mqttc.publish(mqttTopic, message, qos=1)
try:
    while True:
        # Read a line of data from the hc-5 bluetooth module
        incomingData = incomingSer.readline().decode().strip()
        # Process the received data
        parseAndpublish(incomingData)
        
except KeyboardInterrupt:
    # Close the  port when the script is interrupted (e.g., by pressing Ctrl+C)
    incomingSer.close()
    mqttc.disconnect()
    mqttc.loop_stop()


 Sensor Message:  IR=768, BPM=0.00, Avg BPM=0%  Temperature: 26.00°C , NO STRESS

 Sensor Message:  IR=113153, BPM=0.00, Avg BPM=0%  Temperature: 26.00°C , NO STRESS

 Sensor Message:  IR=113204, BPM=0.00, Avg BPM=0%  Temperature: 26.00°C , NO STRESS

 Sensor Message:  IR=113182, BPM=7.49, Avg BPM=0%  Temperature: 26.00°C , NO STRESS

 Sensor Message:  IR=113150, BPM=38.96, Avg BPM=9%  Temperature: 26.00°C , NO STRESS

 Sensor Message:  IR=113192, BPM=66.52, Avg BPM=36%  Temperature: 26.00°C , NO STRESS

 Published Message:  Heart Rate: 66.52, Temperature: 26.0, Stress: NO STRESS

 Sensor Message:  IR=113010, BPM=31.25, Avg BPM=44%  Temperature: 26.00°C , NO STRESS

 Sensor Message:  IR=112966, BPM=142.52, Avg BPM=68%  Temperature: 26.00°C , STRESS

 Published Message:  Heart Rate: 142.52, Temperature: 26.0, Stress: STRESS

 Sensor Message:  IR=904, BPM=142.52, Avg BPM=68%  Temperature: 26.00°C , STRESS

 Published Message:  Heart Rate: 142.52, Temperature: 26.0, Stress: STRESS

 Sens